
# PUC-RIO
## Pós-Graduação em Ciência de Dados e Analytics
---
**Projeto:** Pipeline de Dados BACEN (Camadas Bronze, Silver, Gold)

**Aluno:** Rosângela Gomes André

**Disciplina:** Engenharia de Dados

**Título do Projeto:** Desenvolvimento de um Pipeline de Dados End-to-End para Monitoramento de Remuneração de Servidores Aposentados do BACEN via Databricks.

**Data:** 21 de Dezembro de 2025

## 1. Introdução
Este trabalho representa a implementação de um pipeline de dados end-to-end desenvolvido na plataforma Databricks, com o objetivo de demonstrar a aplicação prática de conceitos de Engenharia de Dados, Modelagem Dimensional e Governança em ambiente de nuvem.

O meu pipeline consiste em cinco etapas bem definidas: **buscar, coletar, modelar, carregar e analisar** os dados. O foco principal deste MVP é superar barreiras técnicas — como a descompactação de grandes volumes de arquivos e a conversão de padrões monetários — utilizando a **Arquitetura Medallion (Bronze, Silver e Gold)** para construir um repositório confiável que permita a análise da folha de pagamento do órgão escolhido.

## 2. Objetivo e Busca dos Dados

### O Problema que pretendo resolver:
Ao explorar os dados públicos, percebi que os arquivos de remuneração de servidores são disponibilizados em formatos de difícil consumo imediato. Para este MVP, propus-me a construir um pipeline que solucione a falta de padronização monetária e a complexidade de compressão desses arquivos, transformando-os em um repositório estruturado e pronto para análise.

### Busca dos Dados:
Na etapa de busca, selecionei dados abertos disponíveis na web através do site **[Portal da Transparência do Governo Federal](https://portaldatransparencia.gov.br/)**. Dentre as diversas bases de dados, escolhi como objeto de estudo as informações de servidores aposentados do Banco Central do Brasil (BACEN). Defini como recorte temporal o período de Janeiro de 2025 até Setembro de 2025.

### As perguntas que desejo responder:
Para validar a eficácia do meu modelo, busquei respostas para as seguintes questões:
1. Qual a média da remuneração bruta total dos aposentados do BACEN?
2. Quais são os 5 cargos com o maior número de servidores aposentados na base?
3. Qual o valor total pago em verbas indenizatórias no período analisado?
4. Como está distribuída a quantidade de servidores por tipo de vínculo?
5. Qual o valor máximo de remuneração líquida encontrado no conjunto de dados?

## 3. Coleta e Armazenamento (Camada Bronze)
A segunda etapa consistiu em coletar esses dados diretamente do site e importá-los para a plataforma Databricks.

* **Processo:** Desenvolvi um robô de coleta em Python (utilizando a biblioteca requests) para realizar o download automatizado de arquivos compactados (.zip).
* **Ingestão Segura:** Para superar restrições de acesso ao sistema de arquivos local do cluster, os arquivos foram processados via fluxo de memória e persistidos em Volumes do Unity Catalog no Databricks.
* **Armazenamento:** Os dados brutos foram mantidos em seu formato original para garantir a auditabilidade da linhagem e a integridade da fonte.

## 4. Modelagem e Carga (ETL)
Para a estruturação dos dados, utilizei a Arquitetura Medallion, organizando o fluxo em três camadas:

* **Bronze:** Onde realizei o armazenamento dos arquivos brutos conforme extraídos da fonte.
* **Silver:** Etapa onde realizei a limpeza e padronização. O maior desafio técnico foi a unificação de **27 pastas de dados históricos** utilizando a técnica unionByName para lidar com a conciliação de esquemas e variações de nomes de colunas (como mes vs mes_referencia).
* **Gold:** Camada de consumo final onde implementei o Modelo Estrela. Separei os dados em tabelas de Dimensão (Servidor, Vínculo e Calendário) e uma tabela de Fato (Remuneração), utilizando hashes SHA256 para criar chaves substitutas (Surrogate Keys), garantindo performance e governança.



## 5. Análise e Solução do Problema

### a. Qualidade de Dados
Realizei uma auditoria de qualidade em cada atributo para garantir que os problemas do conjunto de dados não afetassem os resultados:
* **Tratamento de Encoding:** Identifiquei codificações de caracteres incorretas (ISO-8859-1) que causavam erros em nomes acentuados, resolvendo o problema na leitura inicial.
* **Padronização Monetária:** As remunerações vieram em formato de texto (padrão PT-BR). Implementei uma lógica de limpeza via expressões regulares para converter separadores de milhar e decimais para o tipo Double.
* **Validação de Domínio:** Executei testes de valores mínimos e máximos para assegurar a ausência de remunerações nulas ou negativas que invalidassem as médias estatísticas.

### b. Solução do Problema e Discussão
Através de consultas SQL sobre o modelo dimensional, obtive os seguintes resultados:
* **Análise de Remuneração:** Consegui calcular a média bruta e líquida, identificando que cargos com maiores verbas indenizatórias possuem uma variação menor no líquido devido à natureza não tributável dessas verbas.
* **Distribuição por Cargo:** Identifiquei os 5 cargos com maior volume de beneficiários, facilitando a visão de impacto orçamentário por categoria funcional.
* **Estabilidade do Quadro:** A contagem distinta de servidores por período revelou a taxa de entrada de novos aposentados no regime do BACEN.

## 6. Autoavaliação e Discussão Final
Ao concluir este pipeline, considero que atingi plenamente os objetivos propostos. O uso da Arquitetura Medallion e do Modelo Estrela provou ser a escolha correta, pois transformou uma massa de dados bruta e "suja" (espalhada em 27 pastas) em um catálogo de dados governado.

**Dificuldades:** A principal dificuldade foi lidar com a mudança de esquema entre os meses e o tratamento de caracteres especiais. 
**Trabalhos Futuros:** Para o meu portfólio, pretendo enriquecer este problema implementando testes automatizados de qualidade de dados e conectando a camada Gold a ferramentas de visualização externa como Power BI. O projeto demonstrou que a estruturação de um Data Warehouse moderno no Databricks é essencial para a transparência pública.

## Etapa 1 - Paraparação para das camadas

In [0]:

%sql

-- Cria o catálogo principal para todos os seus dados do BACEN
CREATE CATALOG IF NOT EXISTS catalog_bacen
COMMENT 'Catálogo para dados públicos dos Servidores Aposentados Ativos do Banco do Brasil (BACEN).';

-- Opcional: Crie os esquemas (databases) Bronze, Silver e Gold dentro do novo catálogo
CREATE SCHEMA IF NOT EXISTS catalog_bacen.bronze;
CREATE SCHEMA IF NOT EXISTS catalog_bacen.silver;
CREATE SCHEMA IF NOT EXISTS catalog_bacen.gold;

-- 1. Criação do Volume para armazenar os arquivos
-- Temos o nome 'dados_servidores' para o Volume que armazenará seus arquivos.
CREATE VOLUME IF NOT EXISTS catalog_bacen.bronze.dados_servidores
COMMENT 'Volume para arquivos CSV brutos dos Servidores Aposentados Ativos do Banco do Brasil (BACEN).';

-- 2. Criação do Volume para armazenar as tabelas 
-- Temos o nome 'dados_servidores' para o Volume que armazenará suas tabelas.
CREATE VOLUME IF NOT EXISTS catalog_bacen.silver.dados_servidores
COMMENT 'Volume para armazenar as tabelas dos Servidores Aposentados Ativos do Banco do Brasil (BACEN).';

-- 2. Criação do Volume para armazenar o modelo estrela
-- Temos o nome 'assets' para o Volume que armazenará o modelo estrela.
CREATE VOLUME IF NOT EXISTS catalog_bacen.gold.assets
COMMENT 'Volume para armazenar o modelo estrela dos Servidores Aposentados Ativos do Banco do Brasil (BACEN).';

-- Define o catálogo recém-criado como o padrão para as próximas operações
USE CATALOG catalog_bacen;

## Etapa 2 - Camada Bronze

In [0]:
# Tente rodar esta linha em uma célula separada antes do código principal
%pip install unidecode

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import requests
import os
from zipfile import ZipFile
import shutil
import pandas as pd
import re
from unidecode import unidecode 
from pyspark.sql.functions import col

# --- CONFIGURAÇÃO DE CAMINHOS ---
UC_VOLUME_PATH = "/Volumes/catalog_bacen/bronze/dados_servidores/"

# --- FUNÇÃO DE LIMPEZA ---
def limpar_nome_coluna(nome_coluna):
    nome_limpo = unidecode(nome_coluna) 
    nome_limpo = re.sub(r' \(R\$\)(\(\*\))?$', '_REAIS', nome_limpo)
    nome_limpo = re.sub(r' \(U\$\)(\(\*\))?$', '_DOLAR', nome_limpo)
    nome_limpo = re.sub(r'[/,;{}\(\)\n\t\=–]', ' ', nome_limpo)
    nome_limpo = nome_limpo.strip().lower() 
    nome_limpo = re.sub(r'\s+', '_', nome_limpo)
    nome_limpo = nome_limpo.strip('_')
    nome_limpo = re.sub(r'__+', '_', nome_limpo)
    return nome_limpo

# --- LOOP PARA OS MESES (JANEIRO A SETEMBRO) ---
# Geração de uma lista de meses formatados: ['202501', '202502', ..., '202509']
meses_para_processar = [f"2025{str(mes).zfill(2)}" for mes in range(1, 10)]

print(f"Iniciando processamento para os meses: {meses_para_processar}")

for competencia in meses_para_processar:
    print(f"\n" + "="*50)
    print(f"PROCESSANDO COMPETÊNCIA: {competencia}")
    print("="*50)

    # Dinamiza as variáveis com base no mês
    ZIP_URL = f"https://portaldatransparencia.gov.br/download-de-dados/servidores/{competencia}_Aposentados_BACEN/"
    ZIP_FILENAME = f"{competencia}_Aposentados_BACEN.zip"
    TEMP_DOWNLOAD_PATH = f"/tmp/{ZIP_FILENAME}"
    TEMP_UNZIP_DIR = f"/tmp/unzip_{competencia}"

    try:
        # A. DOWNLOAD
        print(f" -> Baixando: {ZIP_URL}")
        response = requests.get(ZIP_URL, stream=True)
        
        if response.status_code == 404:
            print(f" [AVISO] Dados para {competencia} não encontrados (404). Pulando...")
            continue
            
        response.raise_for_status()
        
        with open(TEMP_DOWNLOAD_PATH, "wb") as file:
            for chunk in response.iter_content(chunk_size=1024 * 1024):
                file.write(chunk)
                
        # B. DESCOMPACTAR
        os.makedirs(TEMP_UNZIP_DIR, exist_ok=True)
        with ZipFile(TEMP_DOWNLOAD_PATH, 'r') as zip_ref:
            zip_ref.extractall(TEMP_UNZIP_DIR)
        
        # C. PROCESSAR ARQUIVOS
        for filename in os.listdir(TEMP_UNZIP_DIR):
            if filename.lower().endswith('.csv'):
                local_csv_path = os.path.join(TEMP_UNZIP_DIR, filename)
                remote_delta_path = UC_VOLUME_PATH + filename.replace(".csv", "") + "_delta"
                
                print(f"    -> Lendo CSV: {filename}")
                
                # Lendo com tratamento de encoding
                with open(local_csv_path, 'r', encoding='ISO-8859-1') as f:
                    csv_content = f.read()

                temp_csv_file_on_volume = f"{UC_VOLUME_PATH}temp_{competencia}_{filename}"
                dbutils.fs.put(temp_csv_file_on_volume, csv_content, overwrite=True)
                
                # Spark Read
                df_temp = (spark.read 
                           .format("csv") 
                           .option("header", "true") 
                           .option("delimiter", ";") 
                           .option("encoding", "ISO-8859-1") 
                           .load(temp_csv_file_on_volume)
                          )
                
                # Renomear Colunas
                mapeamento = {c: limpar_nome_coluna(c) for c in df_temp.columns}
                df_limpo = df_temp
                for old_name, new_name in mapeamento.items():
                    df_limpo = df_limpo.withColumnRenamed(old_name, new_name)
                
                # Salvar Delta
                df_limpo.write.format("delta").mode("overwrite").save(remote_delta_path)
                dbutils.fs.rm(temp_csv_file_on_volume)
                print(f"    -> Sucesso: {remote_delta_path}")

        # D. LIMPEZA TEMPORÁRIA (por mês)
        os.remove(TEMP_DOWNLOAD_PATH) 
        shutil.rmtree(TEMP_UNZIP_DIR)
        print(f" -> Concluído com sucesso para {competencia}")

    except Exception as e:
        print(f" [ERRO] Falha ao processar {competencia}: {e}")
        # Continua para o próximo mês mesmo se um falhar
        continue

print("\nPipeline finalizado para todos os meses disponíveis.")

Iniciando processamento para os meses: ['202501', '202502', '202503', '202504', '202505', '202506', '202507', '202508', '202509']

PROCESSANDO COMPETÊNCIA: 202501
 -> Baixando: https://portaldatransparencia.gov.br/download-de-dados/servidores/202501_Aposentados_BACEN/
    -> Lendo CSV: 202501_Remuneracao.csv
Wrote 1722463 bytes.
    -> Sucesso: /Volumes/catalog_bacen/bronze/dados_servidores/202501_Remuneracao_delta
    -> Lendo CSV: 202501_Observacoes.csv
Wrote 59 bytes.
    -> Sucesso: /Volumes/catalog_bacen/bronze/dados_servidores/202501_Observacoes_delta
    -> Lendo CSV: 202501_Cadastro.csv
Wrote 1779125 bytes.
    -> Sucesso: /Volumes/catalog_bacen/bronze/dados_servidores/202501_Cadastro_delta
 -> Concluído com sucesso para 202501

PROCESSANDO COMPETÊNCIA: 202502
 -> Baixando: https://portaldatransparencia.gov.br/download-de-dados/servidores/202502_Aposentados_BACEN/
    -> Lendo CSV: 202502_Cadastro.csv
Wrote 1777220 bytes.
    -> Sucesso: /Volumes/catalog_bacen/bronze/dados_serv

## Etapa 3 - Camada Silver

In [0]:
from pyspark.sql.functions import col, regexp_replace, to_date, current_timestamp, lit, concat, lpad
from pyspark.sql import functions as F

# --- CONFIGURAÇÃO DE CAMINHOS ---
UC_VOLUME_PATH = "/Volumes/catalog_bacen/bronze/dados_servidores/"
PATH_SILVER = "/Volumes/catalog_bacen/silver/dados_servidores"

print(f"1. Localizando tabelas na Bronze...")

# Lista todos os arquivos/pastas no volume
files = dbutils.fs.ls(UC_VOLUME_PATH)

# Filtra apenas as pastas que terminam com '_delta'
paths_to_load = [f.path for f in files if f.path.endswith("_delta/") or "_delta" in f.path]

if not paths_to_load:
    raise Exception(f"Nenhuma tabela Delta encontrada em {UC_VOLUME_PATH}. Verifique se a camada Bronze rodou com sucesso.")

print(f"   -> Encontradas {len(paths_to_load)} pastas. Carregando e unificando...")

# Lê e unifica as tabelas
df_bronze = spark.read.format("delta").load(paths_to_load[0])

for path in paths_to_load[1:]:
    df_proximo = spark.read.format("delta").load(path)
    df_bronze = df_bronze.unionByName(df_proximo, allowMissingColumns=True)

# --- 2. TRATAMENTO DINÂMICO DE COLUNAS ---
colunas_disponiveis = df_bronze.columns
print(f"🔍 Colunas detectadas: {colunas_disponiveis}")

# Identifica qual coluna de data usar
col_data_origem = None
for candidata in ["mes_referencia", "mes", "MES_REFERENCIA", "MES"]:
    if candidata in colunas_disponiveis:
        col_data_origem = candidata
        break

if not col_data_origem:
    raise Exception("Não foi possível encontrar uma coluna de data (mes ou mes_referencia) nos dados.")

print(f"   -> Usando a coluna '{col_data_origem}' para gerar a data de referência.")

# --- 3. TRANSFORMAÇÕES (LIMPEZA FINANCEIRA) ---
print("2. Iniciando limpeza financeira...")
cols_financeiras = [c for c in df_bronze.columns if '_reais' in c or '_dolar' in c]

df_silver = df_bronze

for c in cols_financeiras:
    # Tratamento para ter o Double corretamente. Ex: converter "1.234,56" em 1234.56 (Double)
    df_silver = df_silver.withColumn(
        c, 
        F.regexp_replace(F.regexp_replace(F.col(c), r'\.', ''), ',', '.').cast("double")
    )

# --- 4. DATA E AUDITORIA (CORREÇÃO DO CANNOT_PARSE_TIMESTAMP) ---
print("3. Tratando campos de data e auditoria...")

# 1. lpad garante que o mês tenha 2 dígitos (ex: '2' vira '02')
# 2. Concatena '01/' + mês + '/' + ano (coluna detectada no seu print)
df_silver = df_silver.withColumn(
    "data_referencia", 
    F.to_date(
        F.concat(
            F.lit("01/"), 
            F.lpad(F.col(col_data_origem), 2, "0"), 
            F.lit("/"), 
            F.col("ano")
        ), 
        "dd/MM/yyyy"
    )
).withColumn("etl_load_timestamp", F.current_timestamp())

# --- 5. GRAVAÇÃO ---
print(f"4. Gravando camada Silver consolidada em: {PATH_SILVER}")

# Cria o diretório silver se não existir
dbutils.fs.mkdirs("/Volumes/catalog_bacen/silver/dados_servidores/")

# Gravação na camada silver
df_silver.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(PATH_SILVER)

print("✅ Camada Silver pronta e validada com sucesso!")

1. Localizando tabelas na Bronze...
   -> Encontradas 27 pastas. Carregando e unificando...
🔍 Colunas detectadas: ['id_servidor_portal', 'nome', 'cpf', 'matricula', 'cod_tipo_aposentadoria', 'tipo_aposentadoria', 'data_aposentadoria', 'descricao_cargo', 'cod_uorg_lotacao', 'uorg_lotacao', 'cod_org_lotacao', 'org_lotacao', 'cod_orgsup_lotacao', 'orgsup_lotacao', 'cod_tipo_vinculo', 'tipo_vinculo', 'situacao_vinculo', 'regime_juridico', 'jornada_de_trabalho', 'data_ingresso_cargofuncao', 'data_nomeacao_cargofuncao', 'data_ingresso_orgao', 'documento_ingresso_servicopublico', 'data_diploma_ingresso_servicopublico', 'diploma_ingresso_cargofuncao', 'diploma_ingresso_orgao', 'diploma_ingresso_servicopublico', 'ano', 'mes', 'observacao', 'remuneraaao_basica_bruta_reais', 'remuneraaao_basica_bruta_dolar', 'abate-teto_reais', 'abate-teto_dolar', 'gratificaaao_natalina_reais', 'gratificaaao_natalina_dolar', 'abate-teto_da_gratificaaao_natalina_reais', 'abate-teto_da_gratificaaao_natalina_dolar',

## Etapa 4 - Camada Gold

In [0]:
from pyspark.sql.functions import col, sha2, concat_ws, year, month, quarter, min, max, count, array_distinct, collect_list
from pyspark.sql import functions as F

# 1. CARREGAR DADOS DA SILVER
print("Lendo dados da Camada Silver...")
df_silver = spark.read.format("delta").load("/Volumes/catalog_bacen/silver/dados_servidores")

# --- DEFINIÇÃO DE METADADOS DE LINHAGEM ---
linhagem_tecnica = (
    "Origem: Portal da Transparência (BACEN - Servidores Aposentados). "
    "Técnica: Ingestão via Python (Requests), descompactação local, conversão para Delta na Bronze. "
    "Transformação: Limpeza de caracteres monetários e tipagem em Silver. "
    "Modelagem: Star Schema com Surrogate Keys (SHA256) na Gold."
)

# --- 2. CRIAÇÃO DAS TABELAS DIMENSIONAIS ---
print("Construindo Dimensões...")

# DIM SERVIDOR
dim_servidor = df_silver.select(
    sha2(col("cpf"), 256).alias("sk_servidor"),
    col("nome"),
    col("cpf"),
    col("matricula"),
    col("descricao_cargo").alias("cargo"),
    col("org_lotacao").alias("orgao"),
    col("uorg_lotacao").alias("unidade_organizacional")
).distinct()

# DIM VÍNCULO
dim_vinculo = df_silver.select(
    sha2(concat_ws("||", col("tipo_vinculo"), col("regime_juridico"), col("jornada_de_trabalho")), 256).alias("sk_vinculo"),
    col("tipo_vinculo"),
    col("regime_juridico"),
    col("jornada_de_trabalho"),
    col("situacao_vinculo")
).distinct()

# DIM CALENDÁRIO
dim_calendario = df_silver.select("data_referencia").distinct() \
    .withColumn("ano", year(col("data_referencia"))) \
    .withColumn("mes", month(col("data_referencia"))) \
    .withColumn("trimestre", quarter(col("data_referencia")))

# --- 3. CRIAÇÃO DA TABELA FATO ---
print("Construindo Tabela Fato...")
fato_remuneracao = df_silver.select(
    sha2(col("cpf"), 256).alias("sk_servidor"),
    sha2(concat_ws("||", col("tipo_vinculo"), col("regime_juridico"), col("jornada_de_trabalho")), 256).alias("sk_vinculo"),
    col("data_referencia"),
    col("remuneraaao_basica_bruta_reais").alias("valor_bruto"),
    col("irrf_reais").alias("valor_irrf"),
    col("pss_rpgs_reais").alias("valor_pss"),
    col("total_de_verbas_indenizatarias_reais").alias("valor_indenizacoes"),
    col("remuneraaao_apas_deduaaes_obrigatarias_reais").alias("valor_liquido")
)

# --- 4. GRAVAÇÃO E REGISTRO NO CATÁLOGO (GOLD) ---
print("Registrando tabelas no Unity Catalog...")
spark.sql("CREATE SCHEMA IF NOT EXISTS catalog_bacen.gold")

dim_servidor.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("catalog_bacen.gold.dim_servidor")
dim_vinculo.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("catalog_bacen.gold.dim_vinculo")
dim_calendario.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("catalog_bacen.gold.dim_calendario")
fato_remuneracao.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("catalog_bacen.gold.fato_remuneracao")

# --- 5. MAIS METADADOS ---
print("Aplicando descrições detalhadas e linhagem...")

# Descrições de Nível de Tabela
spark.sql(f"COMMENT ON TABLE catalog_bacen.gold.fato_remuneracao IS '{linhagem_tecnica}'")
spark.sql("COMMENT ON TABLE catalog_bacen.gold.dim_servidor IS 'Dados cadastrais e identificação dos servidores aposentados do BACEN.'")
spark.sql("COMMENT ON TABLE catalog_bacen.gold.dim_vinculo IS 'Características do vínculo empregatício e regime de trabalho.'")
spark.sql("COMMENT ON TABLE catalog_bacen.gold.dim_calendario IS 'Suporte a análises temporais (Ano, Mês, Trimestre).'")

# Dicionário Fato Remuneração
desc_fato = {
    "sk_servidor": "Chave Surrogate (SHA256 do CPF). Domínio: Hash Único.",
    "sk_vinculo": "Chave Surrogate (vínculo/regime/jornada). Domínio: Hash de contexto.",
    "data_referencia": "Data do fechamento da folha. Domínio: AAAA-MM-DD.",
    "valor_bruto": "Remuneração básica bruta total. Domínio: Numérico Positivo.",
    "valor_irrf": "Imposto de Renda retido. Domínio: Numérico Negativo/Zero.",
    "valor_pss": "Contribuição PSS. Domínio: Numérico Negativo/Zero.",
    "valor_indenizacoes": "Soma de verbas indenizatórias. Domínio: Numérico Positivo.",
    "valor_liquido": "Remuneração líquida final. Domínio: Numérico Positivo."
}
for c, com in desc_fato.items(): spark.sql(f"ALTER TABLE catalog_bacen.gold.fato_remuneracao ALTER COLUMN {c} COMMENT '{com}'")

# Dicionário Dim Servidor
desc_serv = {
    "sk_servidor": "Chave primária da dimensão (Hash do CPF).",
    "nome": "Nome completo do servidor.",
    "cpf": "CPF original (Dado Sensível).",
    "matricula": "Matrícula funcional no BACEN.",
    "cargo": "Último cargo ocupado.",
    "orgao": "Órgão de lotação (BACEN).",
    "unidade_organizacional": "Unidade específica (UORG)."
}
for c, com in desc_serv.items(): spark.sql(f"ALTER TABLE catalog_bacen.gold.dim_servidor ALTER COLUMN {c} COMMENT '{com}'")

# Dicionário Dim Vínculo
desc_vinc = {
    "sk_vinculo": "Chave primária da dimensão vínculo.",
    "tipo_vinculo": "Categoria do vínculo (ex: Estatutário).",
    "regime_juridico": "Regime legal de trabalho.",
    "jornada_de_trabalho": "Carga horária semanal.",
    "situacao_vinculo": "Status do vínculo (Aposentado)."
}
for c, com in desc_vinc.items(): spark.sql(f"ALTER TABLE catalog_bacen.gold.dim_vinculo ALTER COLUMN {c} COMMENT '{com}'")

# Dicionário Dim Calendário
desc_cal = {
    "data_referencia": "Chave primária da dimensão tempo. Domínio: AAAA-MM-DD.",
    "ano": "Ano civil da referência.",
    "mes": "Mês civil (1-12).",
    "trimestre": "Trimestre civil (1-4)."
}
for c, com in desc_cal.items(): spark.sql(f"ALTER TABLE catalog_bacen.gold.dim_calendario ALTER COLUMN {c} COMMENT '{com}'")

# --- 6. ANÁLISE DE DOMÍNIOS E QUALIDADE ---
print("\n--- CATÁLOGO TÉCNICO: ANÁLISE DE DOMÍNIOS ---")
fato_remuneracao.select(
    min("valor_bruto").alias("Min_Bruto"),
    max("valor_bruto").alias("Max_Bruto"),
    min("valor_liquido").alias("Min_Liquido"),
    max("valor_liquido").alias("Max_Liquido")
).show()

print("✅ Camada Gold finalizada e documentada no Unity Catalog.")

Lendo dados da Camada Silver...
Construindo Dimensões...
Construindo Tabela Fato...
Registrando tabelas no Unity Catalog...
Aplicando descrições detalhadas e linhagem...

--- CATÁLOGO TÉCNICO: ANÁLISE DE DOMÍNIOS ---
+---------+---------+-----------+-----------+
|Min_Bruto|Max_Bruto|Min_Liquido|Max_Liquido|
+---------+---------+-----------+-----------+
|   983.41| 46256.98|  -55761.27|  110134.64|
+---------+---------+-----------+-----------+

✅ Camada Gold finalizada e 100% documentada no Unity Catalog.


## Etapa 4 - Camada Gold - Dicionário de Dados Consolidado

In [0]:
from pyspark.sql.functions import lit, col

print("Gerando Dicionário de Dados Consolidado...")

def extrair_metadados(tabela):
    # Comando SQL para ler as colunas e comentários registrados
    return spark.sql(f"DESCRIBE TABLE {tabela}") \
        .select(
            lit(tabela).alias("Nome_Tabela"),
            col("col_name").alias("Nome_Coluna"),
            col("data_type").alias("Tipo_Dado"),
            col("comment").alias("Descricao_Dicionario")
        )

# Lista das suas tabelas Gold
tabelas = [
    "catalog_bacen.gold.fato_remuneracao",
    "catalog_bacen.gold.dim_servidor",
    "catalog_bacen.gold.dim_vinculo",
    "catalog_bacen.gold.dim_calendario"
]

# Une todas as descrições em um único relatório
df_relatorio_final = extrair_metadados(tabelas[0])
for t in tabelas[1:]:
    df_relatorio_final = df_relatorio_final.union(extrair_metadados(t))

# Exibe na tela de forma organizada
display(df_relatorio_final)

Gerando Dicionário de Dados Consolidado...


Nome_Tabela,Nome_Coluna,Tipo_Dado,Descricao_Dicionario
catalog_bacen.gold.fato_remuneracao,sk_servidor,string,Chave Surrogate (SHA256 do CPF). Domínio: Hash Único.
catalog_bacen.gold.fato_remuneracao,sk_vinculo,string,Chave Surrogate (vínculo/regime/jornada). Domínio: Hash de contexto.
catalog_bacen.gold.fato_remuneracao,data_referencia,date,Data do fechamento da folha. Domínio: AAAA-MM-DD.
catalog_bacen.gold.fato_remuneracao,valor_bruto,double,Remuneração básica bruta total. Domínio: Numérico Positivo.
catalog_bacen.gold.fato_remuneracao,valor_irrf,double,Imposto de Renda retido. Domínio: Numérico Negativo/Zero.
catalog_bacen.gold.fato_remuneracao,valor_pss,double,Contribuição PSS. Domínio: Numérico Negativo/Zero.
catalog_bacen.gold.fato_remuneracao,valor_indenizacoes,double,Soma de verbas indenizatórias. Domínio: Numérico Positivo.
catalog_bacen.gold.fato_remuneracao,valor_liquido,double,Remuneração líquida final. Domínio: Numérico Positivo.
catalog_bacen.gold.dim_servidor,sk_servidor,string,Chave primária da dimensão (Hash do CPF).
catalog_bacen.gold.dim_servidor,nome,string,Nome completo do servidor.


### 4.1. Modelo Lógico Estrela

A estrutura lógica da camada **Gold** foi desenhada para otimizar a performance analítica. Abaixo descrevo as entidades e seus relacionamentos:

#### **Tabela de Fatos: fato_remuneracao**
É a tabela central que armazena as métricas quantitativas (fatos) do domínio financeiro.
* **sk_remuneracao (PK):** Chave primária substituta.
* **sk_servidor (FK):** Chave estrangeira para dim_servidor.
* **sk_vinculo (FK):** Chave estrangeira para dim_vinculo.
* **data_referencia (FK):** Chave estrangeira para dim_calendario.
* **valor_bruto (Métrica):** Valor total da remuneração.
* **valor_liquido (Métrica):** Valor após descontos.
* **valor_indenizacoes (Métrica):** Valores extras não tributáveis.

#### **Tabela de Dimensão: dim_servidor**
Contém os atributos descritivos do servidor.
* **sk_servidor (PK):** Surrogate Key gerada via SHA256.
* **nome_servidor:** Nome completo (tratado).
* **cargo:** Nome do cargo ocupado no BACEN.
* **cpf_anonimizado:** Identificação protegida.

#### **Tabela de Dimensão: dim_vinculo**
Descreve a natureza jurídica do trabalho.
* **sk_vinculo (PK):** Surrogate Key.
* **tipo_vinculo:** Ex: Estatutário, Comissionado.
* **jornada_trabalho:** Carga horária semanal.

#### **Tabela de Dimensão: dim_calendario**
Permite a análise temporal dos dados.
* **data_referencia (PK):** Data padronizada (YYYY-MM-DD).
* **ano:** Ano extraído da fonte.
* **mes:** Mês extraído e padronizado.

---
**Diagrama de Relacionamento:**

dim_servidor (1) <---> (N) fato_remuneracao

dim_vinculo (1) <---> (N) fato_remuneracao

dim_calendario (1) <---> (N) fato_remuneracao

---


<div style="text-align: center;">
  <img src="/Volumes/catalog_bacen/gold/assets/modelo_logico_estrela.png" width="600">
  <p><strong>Figura 1:</strong> Modelo Lógico Estrela da Camada Gold.</p>
</div>

### 4.2. Camada Gold - Validação do Modelo Estrela

In [0]:
%sql
SELECT 
    d_cal.ano,
    d_cal.mes,
    d_serv.cargo,
    COUNT(f.sk_servidor) AS total_servidores,
    ROUND(AVG(f.valor_bruto), 2) AS media_bruta,
    ROUND(AVG(f.valor_liquido), 2) AS media_liquida,
    ROUND(SUM(f.valor_indenizacoes), 2) AS total_indenizacoes
FROM 
    catalog_bacen.gold.fato_remuneracao f
JOIN 
    catalog_bacen.gold.dim_servidor d_serv ON f.sk_servidor = d_serv.sk_servidor
JOIN 
    catalog_bacen.gold.dim_calendario d_cal ON f.data_referencia = d_cal.data_referencia
GROUP BY 
    d_cal.ano, d_cal.mes, d_serv.cargo
ORDER BY 
    d_cal.ano DESC, d_cal.mes DESC, media_liquida DESC;

ano,mes,cargo,total_servidores,media_bruta,media_liquida,total_indenizacoes
2025,9,PROCURADOR DO BANCO CENTRAL,305,30347.55,23274.47,0.0
2025,9,AUDITOR DO BANCO CENTRAL,14484,29801.05,22292.37,0.0
2025,9,null,5454,28447.47,21384.13,0.0
2025,9,TECNICO DO BANCO CENTRAL,1499,14933.16,12204.5,0.0
2025,8,PROCURADOR DO BANCO CENTRAL,305,30496.45,23329.98,0.0
2025,8,AUDITOR DO BANCO CENTRAL,14504,29783.11,22285.49,0.0
2025,8,null,5458,28435.37,21388.48,0.0
2025,8,TECNICO DO BANCO CENTRAL,1499,14933.16,12199.72,0.0
2025,7,PROCURADOR DO BANCO CENTRAL,304,30491.69,23326.92,0.0
2025,7,AUDITOR DO BANCO CENTRAL,14512,29794.44,22276.99,0.0


## Etapa 5 - Análise e Solução do Problema

### 1. Média de Remuneração

In [0]:
%sql
SELECT 
    ROUND(AVG(valor_bruto), 2) AS media_bruta,
    ROUND(AVG(valor_liquido), 2) AS media_liquida,
    ROUND(AVG(valor_indenizacoes), 2) AS media_indenizacoes
FROM catalog_bacen.gold.fato_remuneracao;

media_bruta,media_liquida,media_indenizacoes
27585.33,22872.42,0.03


Databricks visualization. Run in Databricks to view.

![Média de Remuneração](/Volumes/catalog_bacen/gold/assets/grafico_1.png)
**Figura 1:** Média de Remuneração.



### 2. Top 5 Cargos por Volume

In [0]:
%sql
SELECT 
    cargo, 
    COUNT(sk_servidor) AS total_servidores
FROM catalog_bacen.gold.dim_servidor
GROUP BY cargo
ORDER BY total_servidores DESC
LIMIT 5;

cargo,total_servidores
AUDITOR DO BANCO CENTRAL,14425
null,5445
TECNICO DO BANCO CENTRAL,1490
PROCURADOR DO BANCO CENTRAL,311


Databricks visualization. Run in Databricks to view.

![Top 5 Cargos por Volume](/Volumes/catalog_bacen/gold/assets/grafico_2.png)
**Figura 2:** Top 5 Cargos por Volume.

### 3. Total de Verbas Indenizatórias por Mês

In [0]:
%sql
SELECT 
    c.ano,
    c.mes,
    SUM(f.valor_indenizacoes) AS total_indenizacoes
FROM catalog_bacen.gold.fato_remuneracao f
JOIN catalog_bacen.gold.dim_calendario c ON f.data_referencia = c.data_referencia
GROUP BY c.ano, c.mes
ORDER BY c.ano, c.mes;

ano,mes,total_indenizacoes
2025,1,0.0
2025,2,718.58
2025,3,0.0
2025,4,0.0
2025,5,659.25
2025,6,0.0
2025,7,0.0
2025,8,0.0
2025,9,0.0


Databricks visualization. Run in Databricks to view.

![Total de Verbas Indenizatórias por Mês](/Volumes/catalog_bacen/gold/assets/grafico_3.png)
**Figura 3:** Total de Verbas Indenizatórias por Mês.

### 4. Distribuição por Tipo de Vínculo

In [0]:
%sql
SELECT 
    tipo_vinculo, 
    COUNT(*) AS quantidade
FROM catalog_bacen.gold.dim_vinculo
GROUP BY tipo_vinculo;

tipo_vinculo,quantidade
Aposentadoria,1
null,1


Databricks visualization. Run in Databricks to view.

![Distribuição por Tipo de Vínculo](/Volumes/catalog_bacen/gold/assets/grafico_4.png)

**Figura 4:** Distribuição por Tipo de Vínculo.

### 5. Máximo Líquido e Outliers

In [0]:
%sql
SELECT 
    d.cargo,
    MAX(f.valor_liquido) AS teto_liquido_encontrado
FROM catalog_bacen.gold.fato_remuneracao f
JOIN catalog_bacen.gold.dim_servidor d ON f.sk_servidor = d.sk_servidor
GROUP BY d.cargo
ORDER BY teto_liquido_encontrado DESC
LIMIT 10;

cargo,teto_liquido_encontrado
null,110134.64
AUDITOR DO BANCO CENTRAL,110134.64
TECNICO DO BANCO CENTRAL,53408.66
PROCURADOR DO BANCO CENTRAL,41147.64


Databricks visualization. Run in Databricks to view.

![Máximo Líquido e Outliers](/Volumes/catalog_bacen/gold/assets/grafico_5.png)
**Figura 5:** Máximo Líquido e Outliers.